In [1]:
import setup_django
setup_django.init()

In [3]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv" #pathlib
ratings_path.exists()

True

In [4]:
df = pd.read_csv(ratings_path)

In [5]:
df.head(n=10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [14]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [19]:
current_users = User.objects.all().values_list('id', flat=True)
rating_users = df['userId'].tolist()

In [20]:
missing_user_ids = set(rating_users) - set(current_users)
missing_user_ids

{21,
 22,
 23,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201

In [21]:
for uid in missing_user_ids:
    User.objects.create(
        id = uid,
        username = f"missing-user-{uid}"
    )

In [22]:
import math
from decimal import Decimal

In [25]:
df['value']= df['rating'].apply(lambda x : math.ceil(Decimal(x))) #convert 2.5 to 3 or 2.0 to 2
df.head()

,userId,movieId,rating,timestamp,value
0,1,31,2.5,1260759144,3
1,1,1029,3.0,1260759179,3
2,1,1061,3.0,1260759182,3
3,1,1129,2.0,1260759185,2
4,1,1172,4.0,1260759205,4


In [26]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']

In [27]:
df.head()

,userId,movieId,rating,timestamp,value,user_id,object_id
0,1,31,2.5,1260759144,3,1,31
1,1,1029,3.0,1260759179,3,1,1029
2,1,1061,3.0,1260759182,3,1,1061
3,1,1129,2.0,1260759185,2,1,1129
4,1,1172,4.0,1260759205,4,1,1172


In [28]:
cols = ['user_id', 'value', 'object_id']
transformed_df = df.copy()[cols]

In [32]:
rating_records = transformed_df.to_dict('records')


In [33]:
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

(20, {'ratings.Rating': 20})

In [34]:
from django.contrib.contenttypes.models import ContentType

ctype = ContentType.objects.get(app_label='movies', model='movie')

In [35]:
new_ratings= []
for r in rating_records:
    r['content_type'] = ctype
    new_ratings.append(
        Rating(**r)
    )

Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)

[<Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: 

In [36]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Rating update took 0:00:06 (6.161584377288818s)
